In [1]:
!pip install transformers datasets evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import evaluate
import torch
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, DataCollatorForLanguageModeling, AutoModelForCausalLM, pipeline
from peft import get_peft_model, LoraConfig, TaskType
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model_path = "/content/drive/MyDrive/LLAMA-2-0.99-humor-finetuned-full"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

In [5]:
# Clasificación mediante prompting
classifier = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [6]:
# Clasificación mediante prompting
classifier = pipeline("text-generation", model=model, tokenizer=tokenizer)

def clasificar_humor(textos, classifier):
    resultados = []
    etiquetas_validas = {'humor', 'no humor'}

    prompt_template = """
    Analiza el siguiente texto y clasifícalo ÚNICAMENTE con una de estas etiquetas:
    [humor], [no humor].

    Texto: "{texto}"

    Respuesta (solo la etiqueta, sin explicaciones):""".strip()

    for texto in textos:
        try:
            # Generar prompt
            prompt = prompt_template.format(texto=texto[:3000])

            # Clasificación
            output = classifier(
                prompt,
                max_new_tokens=4,  # Suficiente para las etiquetas
                do_sample=False,
                temperature=0.0,  # Para mayor determinismo
                pad_token_id=50256  # Usar el pad token de LLaMA si es necesario
            )[0]["generated_text"]

            # Procesamiento robusto de la respuesta
            respuesta = output.replace(prompt, "").strip().lower()
            respuesta = respuesta.split()[0] if respuesta else ""
            #print(respuesta)

            # Validar y normalizar etiqueta
            etiqueta = "humor" if "humor" in respuesta else "no humor"

            resultados.append((etiqueta))

        except Exception as e:
            print(f"Error procesando texto: {texto[:50]}... - {str(e)}")
            resultados.append(("error"))

    return resultados

Device set to use cpu


In [7]:
#Uso ejemplo:
textos = ["hombre", "Había un man tan de malas que hasta Rexona lo abandonó."]
resultados = clasificar_humor(textos, classifier)
resultados

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


['no humor', 'humor']

In [9]:
df_stress = pd.read_excel('stress_data_1.xlsx')

data = df_stress[['Texto', 'Label']]
# Mapear categorías a números
label_map = {"Humor": 1, "No Humor": 0}
data["label"] = data["Label"].map(label_map)

textos = data['Texto'].to_list()

<ipython-input-9-eecf46081fdb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["label"] = data["Label"].map(label_map)


In [11]:
resultados_1 = clasificar_humor(textos, classifier)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [14]:
df_stress['Pred'] = resultados_1

In [15]:
df_stress

,Ejemplos,Texto,Label,Clase_Contraria,Incoherentes,Repeticion,ofuscamiento,Largo,Corto,Pred
0,1,Hombre le pregunta a su mujer como va a casa,No Humor,SI,NaN,NaN,NaN,NO,SI,humor
1,2,"Señor, ¿usted tiene hijos? —preguntó el niño. ...",No Humor,SI,NaN,NaN,NaN,SI,NO,humor
2,3,Un niño observaba a un hombre mayor en la plaz...,No Humor,SI,NaN,NaN,NaN,SI,NO,no humor
3,4,"Jaimito le hace una pregunta a su madre: Mamá,...",No Humor,SI,NaN,NaN,NaN,SI,NO,humor
4,5,Pregunta pregunta pregunta pregunta pregunta p...,No Humor,SI,NaN,NaN,NaN,NO,SI,humor
...,...,...,...,...,...,...,...,...,...,...
95,96,"Después de la tormenta, siempre llega la calma.",No Humor,SI,NaN,NaN,NaN,NO,SI,humor
96,97,El amor nos impulsa a hacer cosas que nunca cr...,No Humor,SI,NaN,NaN,NaN,NO,SI,no humor
97,98,"El amor puede volver loco a cualquiera, pero t...",No Humor,SI,NaN,NaN,NaN,SI,NO,no humor
98,99,Voy en busca de la paz interior que me permita...,No Humor,SI,NaN,NaN,NaN,NO,SI,no humor


In [19]:
df_stress["Label"] = df_stress["Label"].str.lower()
df_stress

,Ejemplos,Texto,Label,Clase_Contraria,Incoherentes,Repeticion,ofuscamiento,Largo,Corto,Pred
0,1,Hombre le pregunta a su mujer como va a casa,no humor,SI,NaN,NaN,NaN,NO,SI,humor
1,2,"Señor, ¿usted tiene hijos? —preguntó el niño. ...",no humor,SI,NaN,NaN,NaN,SI,NO,humor
2,3,Un niño observaba a un hombre mayor en la plaz...,no humor,SI,NaN,NaN,NaN,SI,NO,no humor
3,4,"Jaimito le hace una pregunta a su madre: Mamá,...",no humor,SI,NaN,NaN,NaN,SI,NO,humor
4,5,Pregunta pregunta pregunta pregunta pregunta p...,no humor,SI,NaN,NaN,NaN,NO,SI,humor
...,...,...,...,...,...,...,...,...,...,...
95,96,"Después de la tormenta, siempre llega la calma.",no humor,SI,NaN,NaN,NaN,NO,SI,humor
96,97,El amor nos impulsa a hacer cosas que nunca cr...,no humor,SI,NaN,NaN,NaN,NO,SI,no humor
97,98,"El amor puede volver loco a cualquiera, pero t...",no humor,SI,NaN,NaN,NaN,SI,NO,no humor
98,99,Voy en busca de la paz interior que me permita...,no humor,SI,NaN,NaN,NaN,NO,SI,no humor


In [28]:
from sklearn.metrics import classification_report

target_names = ['no humor', 'humor']
Y_St = df_stress['Label'].to_list()
y_pred_st = df_stress['Pred'].to_list()

print(classification_report(Y_St, y_pred_st, target_names=target_names))

              precision    recall  f1-score   support

    no humor       0.63      0.94      0.75        50
       humor       0.88      0.44      0.59        50

    accuracy                           0.69       100
   macro avg       0.75      0.69      0.67       100
weighted avg       0.75      0.69      0.67       100



In [31]:
stress_contrary = df_stress[df_stress["Clase_Contraria"]=="SI"]

Y_St = stress_contrary["Label"]
y_pred_st = stress_contrary["Pred"]

print(classification_report(Y_St, y_pred_st, target_names=target_names))

              precision    recall  f1-score   support

    no humor       0.78      0.97      0.87        40
       humor       0.93      0.56      0.70        25

    accuracy                           0.82        65
   macro avg       0.86      0.77      0.78        65
weighted avg       0.84      0.82      0.80        65



In [32]:
stress_incoherent = df_stress[df_stress["Incoherentes"]=="SI"]

Y_St = stress_incoherent["Label"]
y_pred_st = stress_incoherent["Pred"]

print(classification_report(Y_St, y_pred_st, target_names=target_names))

              precision    recall  f1-score   support

    no humor       0.00      0.00      0.00       0.0
       humor       0.00      0.00      0.00      10.0

    accuracy                           0.00      10.0
   macro avg       0.00      0.00      0.00      10.0
weighted avg       0.00      0.00      0.00      10.0



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [36]:
stress_rep = df_stress[df_stress["Repeticion"]=="SI"]

Y_St = stress_rep["Label"]
y_pred_st = stress_rep["Pred"]

print(classification_report(Y_St, y_pred_st, target_names=target_names))

              precision    recall  f1-score   support

    no humor       0.80      0.80      0.80        10
       humor       0.80      0.80      0.80        10

    accuracy                           0.80        20
   macro avg       0.80      0.80      0.80        20
weighted avg       0.80      0.80      0.80        20



In [33]:
stress_ofus = df_stress[df_stress["ofuscamiento"]=="SI"]

Y_St = stress_ofus["Label"]
y_pred_st = stress_ofus["Pred"]

print(classification_report(Y_St, y_pred_st, target_names=target_names))

              precision    recall  f1-score   support

    no humor       0.00      0.00      0.00       0.0
       humor       0.00      0.00      0.00       5.0

    accuracy                           0.00       5.0
   macro avg       0.00      0.00      0.00       5.0
weighted avg       0.00      0.00      0.00       5.0



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [34]:
stress_long = df_stress[df_stress["Largo"]=="SI"]

Y_St = stress_long["Label"]
y_pred_st = stress_long["Pred"]

print(classification_report(Y_St, y_pred_st, target_names=target_names))

              precision    recall  f1-score   support

    no humor       0.67      0.97      0.79        31
       humor       0.80      0.21      0.33        19

    accuracy                           0.68        50
   macro avg       0.73      0.59      0.56        50
weighted avg       0.72      0.68      0.62        50



In [35]:
stress_short = df_stress[df_stress["Corto"]=="SI"]

Y_St = stress_short["Label"]
y_pred_st = stress_short["Pred"]

print(classification_report(Y_St, y_pred_st, target_names=target_names))

              precision    recall  f1-score   support

    no humor       0.57      0.89      0.69        19
       humor       0.90      0.58      0.71        31

    accuracy                           0.70        50
   macro avg       0.73      0.74      0.70        50
weighted avg       0.77      0.70      0.70        50

